In [13]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [96]:
df=pd.read_csv("train6_inNums.csv")

x_train=df.iloc[:-1,3:]
y_train=df.iloc[1:,3:]
df1=pd.read_csv("train6_outNums.csv")



,startTime,Hours,Is_weekend,0,1,2,3,4,5,6,...,71,72,73,74,75,76,77,78,79,80
0,2019/1/1 0:00,0.000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2019/1/1 0:10,0.006944,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2019/1/1 0:20,0.013889,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2019/1/1 0:30,0.020833,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2019/1/1 0:40,0.027778,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [126]:
def res_net_module(inputs,node_num):
    temp=tf.keras.layers.BatchNormalization()(inputs)
    temp=tf.keras.layers.Dense(node_num,activation='relu')(temp)
    temp=tf.keras.layers.Dense(node_num,activation='relu')(temp)
    rs=tf.keras.layers.add([temp, inputs])
    return rs

In [127]:
def inception_module(inputs):
    #temp1=tf.keras.layers.Dense(27,activation='relu')(inputs)
    temp2=tf.keras.layers.Dense(128,activation='relu')(inputs)
    temp2=tf.keras.layers.Dense(128,activation='relu')(temp2)
    temp3=tf.keras.layers.Dense(128,activation='relu')(inputs)
    temp3=tf.keras.layers.Dense(128,activation='relu')(temp3)
    temp3=tf.keras.layers.Dense(128,activation='relu')(temp3)
    #temp4=tf.keras.layers.Dense(32,activation='relu')(inputs)
    rs = tf.keras.layers.concatenate([temp2, temp3])
    return rs

In [128]:
input1 = tf.keras.layers.Input(shape=(81,), name='input1')
input2 = tf.keras.layers.Input(shape=(1,), dtype='float32', name='input2')
input3 = tf.keras.layers.Input(shape=(1,), dtype='float32', name='input3')
layer1=inception_module(input1)
res_net1 =res_net_module(layer1,256)
layer2=inception_module(res_net1)
#layer2=tf.keras.layers.Dense(128,activation='relu',name='layer2')(res_net1)
res_net2 =res_net_module(layer2,256)
concatenate1 = tf.keras.layers.concatenate([input2, input3])
layer3=tf.keras.layers.Dense(64,activation='relu')(concatenate1)
layer4=tf.keras.layers.Dense(64,activation='relu')(layer3)
concatenate = tf.keras.layers.concatenate([res_net2,layer4])
dropout1=tf.keras.layers.Dropout(rate=0.2,noise_shape=None,seed=None)(concatenate)
output1=tf.keras.layers.Dense(81,activation='relu',name='output')(dropout1)
model=tf.keras.models.Model(inputs=[input1,input2,input3], outputs=output1)

In [129]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input1 (InputLayer)             (None, 81)           0                                            
__________________________________________________________________________________________________
dense_258 (Dense)               (None, 128)          10496       input1[0][0]                     
__________________________________________________________________________________________________
dense_256 (Dense)               (None, 128)          10496       input1[0][0]                     
__________________________________________________________________________________________________
dense_259 (Dense)               (None, 128)          16512       dense_258[0][0]                  
__________________________________________________________________________________________________
dense_257 

In [130]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.mean_absolute_error,
    metrics={},
)

In [131]:
model.fit(
    x={'input1':x_train.iloc[:-144,:],'input2':df['Hours'].iloc[:-145],'input3':df['Is_weekend'].iloc[:-145]},
    y=y_train.iloc[:-144,:],
    epochs=1000,
    batch_size=128,
    validation_data=([x_train.iloc[3455:,:],df['Hours'].iloc[3455:3599],df['Is_weekend'].iloc[3455:3599]],y_train.iloc[3455:,:])
)

Train on 3455 samples, validate on 144 samples
Epoch 1/1000
3455/3455 [==============================] - 4s 1ms/step - loss: 54.7511 - val_loss: 32.2725
Epoch 2/1000
3455/3455 [==============================] - 0s 93us/step - loss: 32.3291 - val_loss: 24.1823
Epoch 3/1000
3455/3455 [==============================] - 0s 90us/step - loss: 25.6595 - val_loss: 19.6255
Epoch 4/1000
3455/3455 [==============================] - 0s 90us/step - loss: 22.4075 - val_loss: 18.0685
Epoch 5/1000
3455/3455 [==============================] - 0s 92us/step - loss: 20.5792 - val_loss: 16.0749
Epoch 6/1000
3455/3455 [==============================] - 0s 92us/step - loss: 19.5982 - val_loss: 15.5207
Epoch 7/1000
3455/3455 [==============================] - 0s 91us/step - loss: 18.7936 - val_loss: 16.8547
Epoch 8/1000
3455/3455 [==============================] - 0s 91us/step - loss: 17.9583 - val_loss: 17.9638
Epoch 9/1000
3455/3455 [==============================] - 0s 91us/step - loss: 17.4570 - val_loss:

3455/3455 [==============================] - 0s 92us/step - loss: 12.5154 - val_loss: 12.6656
Epoch 77/1000
3455/3455 [==============================] - 0s 89us/step - loss: 12.3987 - val_loss: 12.9527
Epoch 78/1000
3455/3455 [==============================] - 0s 97us/step - loss: 12.3427 - val_loss: 12.8803
Epoch 79/1000
3455/3455 [==============================] - 0s 92us/step - loss: 12.5700 - val_loss: 12.6766
Epoch 80/1000
3455/3455 [==============================] - 0s 90us/step - loss: 12.4858 - val_loss: 12.9877
Epoch 81/1000
3455/3455 [==============================] - 0s 92us/step - loss: 12.3369 - val_loss: 12.4661
Epoch 82/1000
3455/3455 [==============================] - 0s 90us/step - loss: 12.5188 - val_loss: 13.5248
Epoch 83/1000
3455/3455 [==============================] - 0s 94us/step - loss: 12.5165 - val_loss: 13.0801
Epoch 84/1000
3455/3455 [==============================] - 0s 91us/step - loss: 12.4911 - val_loss: 13.1701
Epoch 85/1000
3455/3455 [=================

3455/3455 [==============================] - 0s 90us/step - loss: 11.8337 - val_loss: 12.9114
Epoch 152/1000
3455/3455 [==============================] - 0s 100us/step - loss: 11.8731 - val_loss: 12.7105
Epoch 153/1000
3455/3455 [==============================] - 0s 93us/step - loss: 11.9336 - val_loss: 12.3040
Epoch 154/1000
3455/3455 [==============================] - 0s 92us/step - loss: 11.7885 - val_loss: 12.9240
Epoch 155/1000
3455/3455 [==============================] - 0s 92us/step - loss: 11.6793 - val_loss: 12.8341
Epoch 156/1000
3455/3455 [==============================] - 0s 94us/step - loss: 11.8042 - val_loss: 13.0960
Epoch 157/1000
3455/3455 [==============================] - 0s 94us/step - loss: 11.7488 - val_loss: 12.7769
Epoch 158/1000
3455/3455 [==============================] - 0s 91us/step - loss: 11.8053 - val_loss: 12.2970
Epoch 159/1000
3455/3455 [==============================] - 0s 92us/step - loss: 11.7182 - val_loss: 12.7071
Epoch 160/1000
3455/3455 [=======

3455/3455 [==============================] - 0s 91us/step - loss: 11.4386 - val_loss: 12.6906
Epoch 227/1000
3455/3455 [==============================] - 0s 99us/step - loss: 11.3938 - val_loss: 12.6564
Epoch 228/1000
3455/3455 [==============================] - 0s 94us/step - loss: 11.3677 - val_loss: 12.4789
Epoch 229/1000
3455/3455 [==============================] - 0s 91us/step - loss: 11.4767 - val_loss: 12.3211
Epoch 230/1000
3455/3455 [==============================] - 0s 91us/step - loss: 11.3710 - val_loss: 13.0422
Epoch 231/1000
3455/3455 [==============================] - 0s 91us/step - loss: 11.3456 - val_loss: 12.6717
Epoch 232/1000
3455/3455 [==============================] - 0s 90us/step - loss: 11.3024 - val_loss: 12.5284
Epoch 233/1000
3455/3455 [==============================] - 0s 91us/step - loss: 11.4346 - val_loss: 13.1638
Epoch 234/1000
3455/3455 [==============================] - 0s 90us/step - loss: 11.2908 - val_loss: 12.6052
Epoch 235/1000
3455/3455 [========

3455/3455 [==============================] - 0s 84us/step - loss: 11.0695 - val_loss: 12.3068
Epoch 302/1000
3455/3455 [==============================] - 0s 85us/step - loss: 11.0492 - val_loss: 12.4483
Epoch 303/1000
3455/3455 [==============================] - 0s 86us/step - loss: 11.0431 - val_loss: 12.1540
Epoch 304/1000
3455/3455 [==============================] - 0s 84us/step - loss: 11.1238 - val_loss: 12.0943
Epoch 305/1000
3455/3455 [==============================] - 0s 86us/step - loss: 11.0998 - val_loss: 12.5895
Epoch 306/1000
3455/3455 [==============================] - 0s 85us/step - loss: 11.0484 - val_loss: 12.3386
Epoch 307/1000
3455/3455 [==============================] - 0s 84us/step - loss: 11.2034 - val_loss: 12.4604
Epoch 308/1000
3455/3455 [==============================] - 0s 84us/step - loss: 11.0481 - val_loss: 12.5827
Epoch 309/1000
3455/3455 [==============================] - 0s 85us/step - loss: 11.0390 - val_loss: 12.8540
Epoch 310/1000
3455/3455 [========

...

3455/3455 [==============================] - 0s 93us/step - loss: 10.3019 - val_loss: 12.6284
Epoch 677/1000
3455/3455 [==============================] - 0s 90us/step - loss: 10.2880 - val_loss: 12.4653
Epoch 678/1000
3455/3455 [==============================] - 0s 89us/step - loss: 10.2596 - val_loss: 12.6473
Epoch 679/1000
3455/3455 [==============================] - 0s 92us/step - loss: 10.3227 - val_loss: 12.1452
Epoch 680/1000
3455/3455 [==============================] - 0s 90us/step - loss: 10.3347 - val_loss: 12.5012
Epoch 681/1000
3455/3455 [==============================] - 0s 91us/step - loss: 10.2678 - val_loss: 12.2722
Epoch 682/1000
3455/3455 [==============================] - 0s 92us/step - loss: 10.3979 - val_loss: 12.5087
Epoch 683/1000
3455/3455 [==============================] - 0s 91us/step - loss: 10.3357 - val_loss: 12.7650
Epoch 684/1000
3455/3455 [==============================] - 0s 92us/step - loss: 10.3367 - val_loss: 12.6721
Epoch 685/1000
3455/3455 [========

3455/3455 [==============================] - 0s 93us/step - loss: 10.3028 - val_loss: 12.7774
Epoch 752/1000
3455/3455 [==============================] - 0s 93us/step - loss: 10.2168 - val_loss: 12.7174
Epoch 753/1000
3455/3455 [==============================] - 0s 91us/step - loss: 10.1145 - val_loss: 12.4447
Epoch 754/1000
3455/3455 [==============================] - 0s 91us/step - loss: 10.1329 - val_loss: 12.7726
Epoch 755/1000
3455/3455 [==============================] - 0s 92us/step - loss: 10.2293 - val_loss: 12.5471
Epoch 756/1000
3455/3455 [==============================] - 0s 92us/step - loss: 10.2509 - val_loss: 12.4397
Epoch 757/1000
3455/3455 [==============================] - 0s 92us/step - loss: 10.2479 - val_loss: 12.6604
Epoch 758/1000
3455/3455 [==============================] - 0s 92us/step - loss: 10.1380 - val_loss: 12.9050
Epoch 759/1000
3455/3455 [==============================] - 0s 92us/step - loss: 10.2100 - val_loss: 12.7049
Epoch 760/1000
3455/3455 [========

3455/3455 [==============================] - 0s 91us/step - loss: 10.2953 - val_loss: 12.5583
Epoch 827/1000
3455/3455 [==============================] - 0s 89us/step - loss: 10.0743 - val_loss: 12.6106
Epoch 828/1000
3455/3455 [==============================] - 0s 88us/step - loss: 10.0599 - val_loss: 12.8761
Epoch 829/1000
3455/3455 [==============================] - 0s 91us/step - loss: 10.0432 - val_loss: 12.4893
Epoch 830/1000
3455/3455 [==============================] - 0s 88us/step - loss: 10.0864 - val_loss: 12.4748
Epoch 831/1000
3455/3455 [==============================] - 0s 88us/step - loss: 10.0584 - val_loss: 12.5500
Epoch 832/1000
3455/3455 [==============================] - 0s 90us/step - loss: 10.0294 - val_loss: 12.7709
Epoch 833/1000
3455/3455 [==============================] - 0s 89us/step - loss: 10.1201 - val_loss: 12.9287
Epoch 834/1000
3455/3455 [==============================] - 0s 89us/step - loss: 10.1306 - val_loss: 12.7698
Epoch 835/1000
3455/3455 [========

3455/3455 [==============================] - 0s 90us/step - loss: 10.0492 - val_loss: 12.8483
Epoch 902/1000
3455/3455 [==============================] - 0s 88us/step - loss: 10.0170 - val_loss: 12.7298
Epoch 903/1000
3455/3455 [==============================] - 0s 89us/step - loss: 9.9761 - val_loss: 12.9828
Epoch 904/1000
3455/3455 [==============================] - 0s 88us/step - loss: 10.0156 - val_loss: 12.4699
Epoch 905/1000
3455/3455 [==============================] - 0s 89us/step - loss: 10.1427 - val_loss: 12.3855
Epoch 906/1000
3455/3455 [==============================] - 0s 89us/step - loss: 10.1319 - val_loss: 12.6695
Epoch 907/1000
3455/3455 [==============================] - 0s 88us/step - loss: 10.0804 - val_loss: 12.5389
Epoch 908/1000
3455/3455 [==============================] - 0s 88us/step - loss: 10.0534 - val_loss: 12.6662
Epoch 909/1000
3455/3455 [==============================] - 0s 89us/step - loss: 9.9016 - val_loss: 12.6437
Epoch 910/1000
3455/3455 [==========

3455/3455 [==============================] - 0s 90us/step - loss: 10.0074 - val_loss: 12.7805
Epoch 977/1000
3455/3455 [==============================] - 0s 89us/step - loss: 9.9370 - val_loss: 12.6805
Epoch 978/1000
3455/3455 [==============================] - 0s 89us/step - loss: 9.8816 - val_loss: 12.7178
Epoch 979/1000
3455/3455 [==============================] - 0s 89us/step - loss: 9.9316 - val_loss: 12.5022
Epoch 980/1000
3455/3455 [==============================] - 0s 89us/step - loss: 9.9024 - val_loss: 12.9004
Epoch 981/1000
3455/3455 [==============================] - 0s 88us/step - loss: 9.9564 - val_loss: 12.6251
Epoch 982/1000
3455/3455 [==============================] - 0s 89us/step - loss: 9.9952 - val_loss: 12.8419
Epoch 983/1000
3455/3455 [==============================] - 0s 89us/step - loss: 9.9986 - val_loss: 12.8005
Epoch 984/1000
3455/3455 [==============================] - 0s 89us/step - loss: 9.9888 - val_loss: 12.5271
Epoch 985/1000
3455/3455 [================